In [ ]:
for epoch in range(30):  # loop over the dataset multiple times
    
    running_loss = 0.0
    
    for (idx, batch) in enumerate(train_loader):
        sentence = batch["Text"]
        inputs = tokenizer(sentence,padding=True,truncation=True,return_tensors="pt")
        inputs = {k:v.to(device) for k,v in inputs.items()}

        # map string labels to class idex
        labels = [label_maps[stringtoId] for stringtoId in (batch['Class'])]

        
        # convert list to tensor
        labels = torch.tensor(labels).unsqueeze(0)
        labels = labels.to(device)


        #(batch_size, seq_len)
        #print(inputs["input_ids"].shape)

         # zero the parameter gradients
        optimizer.zero_grad()

        outputs = model(**inputs,labels=labels,output_hidden_states=True)     
        
        hidden_states = outputs.hidden_states
        
        last_hidden_states = hidden_states[12]
        
        # https://stackoverflow.com/questions/63040954/how-to-extract-and-use-bert-encodings-of-sentences-for-text-similarity-among-sen 
        # (batch_size,seq_len,embed_dim)
        h = last_hidden_states[:,0,:]
        
        # create pair samples
        T, h_i, h_j, idx_yij = create_supervised_pair(h,batch['Class'],debug=False)
        
        if h_i is None:
            print("skip this batch")
            skip_time +=1
            continue
               
        # supervised contrastive loss 
        loss_s_cl = supervised_contrasive_loss(h_i, h_j, h, T,temp=temp,idx_yij=idx_yij,debug=False)
        
        # cross entropy loss
        loss_classify, logits = outputs[:2]
        
        # loss total
        loss = loss_s_cl + (lamda * loss_classify)
        
        loss.backward()
        optimizer.step()
       
        
        print(f'[{epoch + 1}, {idx}] loss_total: {loss.item()}, loss_s_cl:{loss_s_cl.item()}, loss_classify:{lamda * loss_classify.item()}')
       
        clear_output(wait=True)      